In [33]:
""" Preliminary Test for Painting a Point Cloud in Open3D. """

import numpy as np
import open3d as o3d
from collections import deque

def visualize_pcd(pcd: list):
    """
    Visualize it in Open3D interface.
    pcd: list. Must be passed in a list format, if wished multiple
    point clouds can be passe.  
    """
    o3d.visualization.draw_geometries(pcd,
                                    zoom=0.49,
                                    front=[-0.4999, -0.1659, -0.8499],
                                    lookat=[2.1813, 2.0619, 2.0999],
                                    up=[0.1204, -0.9852, 0.1215])

test_data_path = "wall_table_chair.pcd"

pcd = o3d.io.read_point_cloud(test_data_path)

box_points = np.asarray(pcd.points)
print(f"Initial PC Size: {box_points.shape}")

box_points_green = deque()
box_points_red = deque()

# Split the mesh box points in two so that we draw only half of the points
for i in range(0, box_points.shape[0]):
    if i >= box_points.shape[0]/2:
        box_points_red.append(box_points[i])
    else:
        box_points_green.append((box_points[i]))

print(f"Red Box Points Size: {len(box_points_red)}")
print(f"Green Box Points Size: {len(box_points_green)}")

# Create two new objects that we'll assign half of the points
# and then paint in one color
cluster_1 = o3d.geometry.PointCloud()
cluster_1.points = o3d.utility.Vector3dVector(box_points_red)

cluster_2 = o3d.geometry.PointCloud()
cluster_2.points = o3d.utility.Vector3dVector(box_points_green)

cluster_1.paint_uniform_color([1.0, 0, 0])
cluster_2.paint_uniform_color([0.0, 0, 1.0])

visualize_pcd([cluster_1, cluster_2])


Initial PC Size: (40000, 3)
Red Box Points Size: 20000
Green Box Points Size: 20000


In [37]:

from pydantic import BaseModel
from typing import List
import random

class ColoredPointCloudCluster(BaseModel):
    """
    Class that describes data structure to represent a cluster to be colored.
    HDBSCAN will output N clusters, therefore N classes should be created 
    """
    label: int
    color_open3d: List[float]
    data_points: List[float]

class PointCloud(BaseModel):
    clusters: ColoredPointCloudCluster

def get_number_of_labels(labels: list) -> list:
    """
    HDBSCAN will return an array containing the labels, this function creates a unique list out of
    the unique ids.
    """
    return set(labels)

def get_pc_size(pcd) -> int:
    return pcd.shape[0]

def generate_colored_clusters(unique_labels: list) -> list:
    """
    Based on the N unique labels identified from HDBSCAN clustering algorithm, N classes
    will be generated (along with colors) to be populated with the separeted points.
    In other words N different point clouds will be generated and then merged together.
    """
    clusters = []
    for label in unique_labels:
        # Generates Red, Green and Blue color channels for the Cluster color
        cpcc = ColoredPointCloudCluster(
            label = label,
            color_open3d = [
                            round(random.uniform(0.0, 1.0), 3),    # Red channel
                            round(random.uniform(0.0, 1.0), 3),    # Green channel
                            round(random.uniform(0.0, 1.0), 3)],   # Blue channel
            data_points = []
        )
        clusters.append(cpcc)
    return clusters

def assign_data_points_to_colored_cluster(
                                  colored_clusters: List[ColoredPointCloudCluster],
                                  point_cloud,
                                  size_of_pc: int,
                                  unique_labels: List[int], 
                                  labels_from_hdbscan: List[List[float]]
                                  ):
    """
    Compares each data point XYZ and its respective label from the point cloud and HDBSCAN output and
    assigns each data point to the correct cluster (to the correct Class).
    Each class (cluster) will be populated with their respective datapoints, other values are going to be 0. 
    """
    for i in range(size_of_pc):
        for label in unique_labels:
            if labels_from_hdbscan[i] == label:
                # Since we're expecting the labels to be ordered, we can reference the correct class/cluster id by the label itself
                # Stores the datapoint to the class
                colored_clusters[label].data_points.append(point_cloud[i])


def convert_to_open3d_and_paint_clusters(colored_point_cloud_clusters):
    colored_clusters_with_data_points = []
    for colored_point_cloud_cluster in colored_point_cloud_clusters:
        colored_pc = o3d.geometry.PointCloud()
        colored_pc.points = o3d.utility.Vector3dVector(colored_point_cloud_cluster.data_points)
        colored_pc.paint_uniform_color(colored_point_cloud_cluster.color_open3d)
        colored_clusters_with_data_points.append(colored_pc)
    return colored_clusters_with_data_points


# Testing Class Generation
sample_point_cloud = [[0.55, 0.64, 0.8], [1.1, 0.3, 0.6], [1.7, 0.5, 0.9], [0.3, 0.7, 0.9]]
spc = np.asarray(sample_point_cloud)
labels = [0, 1, 1, 0]
pc_size = get_pc_size(spc)
unique_labels = get_number_of_labels(labels)
colored_clusters = generate_colored_clusters(unique_labels)
assign_data_points_to_colored_cluster(colored_clusters, spc, pc_size, unique_labels, labels)
print(colored_clusters[0].data_points)
print(colored_clusters[1].data_points)
open3d_clusterized_colored_pc = convert_to_open3d_and_paint_clusters(colored_clusters)

visualize_pcd(open3d_clusterized_colored_pc)



[array([0.55, 0.64, 0.8 ]), array([0.3, 0.7, 0.9])]
[array([1.1, 0.3, 0.6]), array([1.7, 0.5, 0.9])]
[PointCloud with 2 points., PointCloud with 2 points.]


In [ ]:
def split_point_cloud_in_clusters(pcd):
    """
    Takes in a point cloud that has been clustered with HDBSCAN with N clusters,
    splits the data into N lists and paints each cluster with a different color. 
    """

    # Somehow generate lists dynamically based on the number of clusters
    # And we must assign the points to each list based on their cluster 
    # Use random to generate numbers between 0 and 1.0 to paint for different colors
    
    # Maybe create a class (Data Strucutre) which we will assign the data point and the cluster it belongs to
    # Actually: one big class that comprises a subclass
    # https://realpython.com/factory-method-python/


    sample_point_cloud = [[0.55, 0.64, 0.8], [1.1, 0.3, 0.6], [1.7, 0.5, 0.9], [0.3, 0.7, 0.9]]

    # Simulates the labels identified
    labels = [0, 1, 2, 3]

    number_of_clusters = max(labels) + 1

    # In the loop we should be checking if the current data point belongs to which cluster
    # And assign the data point to a specific and separated created list
    #cluster_2.points = o3d.utility.Vector3dVector(box_points_green)

    #cluster_1.paint_uniform_color([1.0, 0, 0])


    #for label in labels:
        

    """for i in range(0, number_of_clusters):
        for label in labels:
            # how to generate if conditions dynamically here?
            # Some design pattern here will help
            #https://www.youtube.com/watch?v=tv-_1er1mWI
            pass"""